In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)


!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install --upgrade scipy networkx

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 24.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
from torch_geometric.utils.train_test_split_edges import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader

from torch_geometric.datasets import TUDataset

In [ ]:
m = nn.Dropout(p=0)
input = torch.randn(4, 4)
output = m(input)
print(input)
#print(output)

tensor([[-0.5486,  0.8004,  0.3328, -1.8980],
        [ 0.8009,  0.8011,  0.4224, -0.5019],
        [-0.5427,  0.7062, -0.1693, -0.6536],
        [ 0.6186, -0.5346, -1.0154,  0.6742]])


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataset = TUDataset(root='data/TUDataset', name='MUTAG')

Extracting data/TUDataset/MUTAG/MUTAG.zip
Processing...
Done!


In [ ]:

import sys
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
from torch_geometric.typing import Adj, OptPairTensor, Size, SparseTensor
import torch
import torch.nn as nn
import matrix_util as Mat_Util
class GNN_GraphSage_Layer(nn.Module):
    '''
        #    A single GraphSage Layer: Graph Sampling and Aggregate
    '''
    def __init__(self, input_dim, output_dim, Bias, normalize_embedding, dropout, aggregation):
        super(GNN_GraphSage_Layer, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.Bias = Bias
        self.dropout = dropout
        self.normalize_embedding = normalize_embedding
        self.aggregation = aggregation
        #self.add_self = add_self

        if self.aggregation == 'mean':
            self.learnable_weights = nn.Linear(self.input_dim*2, self.output_dim, bias=self.Bias)
        else:
            self.learnable_weights = nn.Linear(self.input_dim, self.output_dim, bias=self.Bias)

        self.normalize = F.normalize

    def forward(self, input_tensor, tilda_adjacency_matrix):

        if self.aggregation == 'mean':
            tilda_adjacency_matrix = tilda_adjacency_matrix / tilda_adjacency_matrix.sum(-1, keepdim=True)
        num_node_per_graph = tilda_adjacency_matrix.size(1)
        print("num_node_per_graph: ", num_node_per_graph)

        tilda_adjacency_matrix_neghborhood = torch.mm(tilda_adjacency_matrix, input_tensor) # Y = A~ * X

        neighborhood_aggregated = torch.cat((tilda_adjacency_matrix_neghborhood, input_tensor), 1)

        node_linear = self.learnable_weights(neighborhood_aggregated) # Y * W

        if self.normalize_embedding:
            node_linear = self.normalize(node_linear, p=2, dim=2)



        return node_linear


In [ ]:
graphsage = GNN_GraphSage_Layer(input_dim=7, output_dim=7, Bias=True, normalize_embedding=True, dropout=0, aggregation='mean')
graph = dataset[0]
tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(graph.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(graph.edge_index).todense())))
#print(adjacency_matrix)

graphsage_output = graphsage(graph.x, tilda_adjacency_matrix)
print(graphsage)

num_node_per_graph:  17
GNN_GraphSage_Layer(
  (learnable_weights): Linear(in_features=14, out_features=7, bias=True)
)


In [ ]:
graphsage = GNN_GraphSage_Layer(input_dim=7, output_dim=7, Bias=True, normalize_embedding=True, dropout=0, aggregation='mean')
graph = dataset[0]
tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(graph.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(graph.edge_index).todense())))
#print(adjacency_matrix)

graphsage_output = graphsage(graph.x, tilda_adjacency_matrix)

num_node_per_graph:  17


In [5]:
import sys
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
from torch_geometric.typing import Adj, OptPairTensor, Size, SparseTensor
import torch
import torch.nn as nn
import matrix_util as Mat_Util
class GNN_Batched_GraphSage_Layer(nn.Module):
    '''
        #    A single GraphSage Layer: Graph Sampling and Aggregate
    '''
    def __init__(self, input_dim, output_dim, Bias, normalize_graphsage, dropout, aggregation, concat):
        super(GNN_Batched_GraphSage_Layer, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.Bias = Bias
        self.dropout = dropout
        self.normalize_graphsage = normalize_graphsage
        self.aggregation = aggregation
        self.concat = concat

        if self.concat:
            self.learnable_weights = nn.Linear(self.input_dim*2, self.output_dim, bias=self.Bias)
        else:
            self.learnable_weights = nn.Linear(self.input_dim, self.output_dim, bias=self.Bias)

        self.normalize = F.normalize


    def forward(self, new_features, tilda_adjacency_matrix):

        #new_features = new_features.to(torch.float32)
        new_features = new_features.type(torch.float32)
        tilda_adjacency_matrix = tilda_adjacency_matrix.type(torch.float32)


        if self.aggregation == 'mean':
            tilda_adjacency_matrix = tilda_adjacency_matrix / (tilda_adjacency_matrix.sum(-2, keepdim=False).unsqueeze(dim=-2))
        tilda_adjacency_matrix = torch.nan_to_num(tilda_adjacency_matrix, nan=0)

        aggregated_neghborhood = torch.bmm(tilda_adjacency_matrix, new_features) # Y = A~ * X

        if self.concat:
            aggregated_neghborhood = torch.cat((aggregated_neghborhood, new_features), 2)

        node_linear = self.learnable_weights(aggregated_neghborhood) # Y * W

        if self.normalize_graphsage:
            node_linear = self.normalize(node_linear, p=2, dim=2)
        node_linear = torch.nan_to_num(node_linear, nan=0)
        #print("node_linear: ",node_linear)

        return node_linear

In [ ]:
mat = torch.arange(24).view(2,3,2, -1)
print("mat: ", mat.size())
print(mat)
print("===============================")
sum = torch.sum(mat, dim=-2).unsqueeze(dim=-2)
print("sum: ", sum.size())
print(sum)
print("===============================")
final = mat/sum
print(final)




mat:  torch.Size([2, 3, 2, 2])
tensor([[[[ 0,  1],
          [ 2,  3]],

         [[ 4,  5],
          [ 6,  7]],

         [[ 8,  9],
          [10, 11]]],


        [[[12, 13],
          [14, 15]],

         [[16, 17],
          [18, 19]],

         [[20, 21],
          [22, 23]]]])
sum:  torch.Size([2, 3, 1, 2])
tensor([[[[ 2,  4]],

         [[10, 12]],

         [[18, 20]]],


        [[[26, 28]],

         [[34, 36]],

         [[42, 44]]]])
tensor([[[[0.0000, 0.2500],
          [1.0000, 0.7500]],

         [[0.4000, 0.4167],
          [0.6000, 0.5833]],

         [[0.4444, 0.4500],
          [0.5556, 0.5500]]],


        [[[0.4615, 0.4643],
          [0.5385, 0.5357]],

         [[0.4706, 0.4722],
          [0.5294, 0.5278]],

         [[0.4762, 0.4773],
          [0.5238, 0.5227]]]])


In [ ]:
t = torch.tensor([[1.,2.,3.]])
print("tensor: ", t)
t1 = F.normalize(t, p=2.0, dim = 0)
t2 = F.normalize(t, p=2.0, dim = 1)
print(t1)
print(t2)

tensor:  tensor([[1., 2., 3.]])
tensor([[1., 1., 1.]])
tensor([[0.2673, 0.5345, 0.8018]])


In [6]:
batched_graphsage_example = GNN_Batched_GraphSage_Layer(input_dim=7, output_dim=7, Bias=True, normalize_graphsage=True, dropout=0,
                                                        aggregation='mean', concat=False)
print(batched_graphsage_example)

GNN_Batched_GraphSage_Layer(
  (learnable_weights): Linear(in_features=7, out_features=7, bias=True)
)


In [7]:
batched_dataset = DataLoader(dataset, batch_size=1, shuffle=False)

for batched_graph in batched_dataset:
    #print(len(batched_graph))
    x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y
    #print(batched_graph.num_graphs)
    tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense())))
    #print("tilda_adjacency_matrix: ", tilda_adjacency_matrix.size())
    softmaxed_output = batched_graphsage_example(batched_graph, tilda_adjacency_matrix)
    print("softmaxed_output: ", softmaxed_output.size())
    break
print(softmaxed_output)

AttributeError: ignored

In [ ]:
print(graphsage_output.size())
print(graphsage_output)

torch.Size([17, 7])
tensor([[ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.5391, -0.0586,  0.2297, -0.1688, -0.6186, -0.4392,  0.2216],
        [ 0.6193, -0.0558,  0.3279, -0.2110, -0.5358, -0.3831,  0.1656],
        [ 0.5391, -0.0586,  0.2

In [ ]:
sum=0
length_list = []
for i in range(len(dataset)):
    sum = sum + len(dataset[i].x)
    length_list.append(len(dataset[i].x))
    #print(len(dataset[i].x))
print("number of nodes in whole dataset: ", sum)
print(length_list)
print("max number of nodes in graphs: ", max(length_list))

number of nodes in whole dataset:  3371
[17, 13, 13, 19, 11, 28, 16, 20, 12, 17, 17, 20, 22, 13, 19, 22, 11, 17, 13, 18, 18, 17, 23, 27, 17, 13, 23, 17, 23, 23, 22, 24, 23, 13, 17, 14, 17, 15, 15, 13, 17, 13, 19, 17, 12, 23, 22, 17, 20, 16, 26, 26, 19, 19, 14, 17, 21, 25, 23, 19, 17, 11, 23, 20, 16, 16, 20, 23, 19, 14, 26, 16, 16, 23, 18, 10, 16, 16, 17, 19, 12, 25, 16, 11, 23, 23, 16, 12, 13, 23, 25, 19, 23, 19, 19, 25, 18, 13, 15, 16, 23, 26, 19, 23, 17, 20, 25, 19, 28, 24, 11, 15, 13, 16, 12, 10, 21, 23, 21, 13, 25, 21, 17, 11, 19, 20, 21, 15, 14, 11, 19, 11, 21, 22, 11, 20, 22, 13, 11, 16, 11, 20, 12, 11, 16, 16, 13, 15, 20, 12, 12, 22, 15, 12, 14, 12, 20, 20, 20, 14, 26, 24, 22, 22, 19, 23, 22, 11, 13, 17, 24, 12, 21, 22, 22, 12, 20, 13, 22, 28, 11, 14, 22, 22, 13, 12, 21, 16]
max number of nodes in graphs:  28


In [ ]:

class DiffPool_Embedding_Layer(nn.Module):
    '''
   #     Z, new features size
    '''
    def __init__(self, input_dim_size, new_feat_dim_size, Bias, normalize_embedding, dropout, aggregation):
        super(DiffPool_Embedding_Layer, self).__init__()
        self.input_dim_size = input_dim_size
        self.new_feat_dim_size = new_feat_dim_size
        self.Bias = Bias
        self.normalize_embedding = normalize_embedding
        self.dropout = dropout
        self.aggregation = aggregation
        self.embedding_layer = GNN_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_feat_dim_size, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        #self.embedding_layer = GNN_Batched_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_feat_dim_size, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        self.act_fun = F.relu


    def forward(self, input_tensor, tilda_adjacency_matrix):
        #x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y
        z_l_init = self.embedding_layer(input_tensor, tilda_adjacency_matrix)
        z_l_init = self.act_fun(z_l_init)

        return z_l_init


In [8]:
import sys
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
from torch_geometric.typing import Adj, OptPairTensor, Size, SparseTensor
import torch
import torch.nn as nn
import matrix_util as Mat_Util
import Batched_GraphSage_Layer as batched_graphsage_layer
class Batched_DiffPool_Embedding_Layer(nn.Module):
    '''
   #     Z, new features size
    '''
    def __init__(self, input_dim, embedding_num_block_layers, hid_dim, embedded_dim, Bias, normalize_graphsage, dropout,
                 aggregation, concat):
        super(Batched_DiffPool_Embedding_Layer, self).__init__()
        self.input_dim = input_dim
        self.embedding_num_block_layers = embedding_num_block_layers
        self.hid_dim = hid_dim
        self.embedded_dim = embedded_dim
        self.Bias = Bias
        self.normalize_graphsage = normalize_graphsage
        self.dropout = dropout
        self.aggregation = aggregation
        self.act_fun = F.relu
        self.concat = concat


        self.DiffPool_Embedding = nn.ModuleList()

        if self.concat:
            self.hid_dim = 2*self.hid_dim

        self.DiffPool_Embedding.append(GNN_Batched_GraphSage_Layer(input_dim=self.input_dim, output_dim=self.hid_dim,
                                                                                           Bias=self.Bias, normalize_graphsage=self.normalize_graphsage,
                                                                                           dropout=self.dropout, aggregation=self.aggregation, concat=self.concat))


        for i in range(embedding_num_block_layers):
            self.DiffPool_Embedding.append(GNN_Batched_GraphSage_Layer(input_dim=self.hid_dim, output_dim=self.hid_dim,
                                                                                               Bias=self.Bias, normalize_graphsage=self.normalize_graphsage,
                                                                                               dropout=self.dropout, aggregation=self.aggregation, concat=self.concat))

        self.DiffPool_Embedding.append(GNN_Batched_GraphSage_Layer(input_dim=self.hid_dim, output_dim=self.embedded_dim,
                                                                                           Bias=self.Bias, normalize_graphsage=self.normalize_graphsage,
                                                                                           dropout=self.dropout, aggregation=self.aggregation, concat=self.concat))



    def forward(self, input_tensor, tilda_adjacency_matrix):

        for layer in self.DiffPool_Embedding:
            input_tensor = self.act_fun(layer(input_tensor, tilda_adjacency_matrix))

        return input_tensor


In [ ]:
diffpool_embedding_example = Batched_DiffPool_Embedding_Layer(input_dim=7, embedding_num_block_layers=1, hid_dim=7, embedded_dim=7, Bias=True,
                                                              normalize_graphsage=True, dropout=0, aggregation="mean", concat=True)

In [ ]:
#import binascii

class DiffPool_Assignment_Layer(nn.Module):
    '''
    #    S, new clusters, new number of nodes
    '''
    def __init__(self, input_dim_size, new_num_nodes, Bias, normalize_embedding, dropout, aggregation):
        super(DiffPool_Assignment_Layer, self).__init__()
        self.input_dim_size = input_dim_size
        self.new_num_nodes = new_num_nodes
        self.Bias = Bias
        self.normalize_embedding = normalize_embedding
        self.dropout = dropout
        self.aggregation = aggregation
        self.assinment_layer = GNN_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_num_nodes, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        #self.assinment_layer = GNN_Batched_GraphSage_Layer(input_dim=self.input_dim_size, output_dim=self.new_num_nodes, Bias=self.Bias, normalize_embedding=self.normalize_embedding, dropout=self.dropout, aggregation=self.aggregation)
        self.act_fun = F.relu

    def forward(self, input_tensor, tilda_adjacency_matrix):
        #x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y
        s_l_init = self.assinment_layer(input_tensor, tilda_adjacency_matrix)
        s_l_init = self.act_fun(s_l_init)

        s_l = F.softmax(s_l_init, dim=-1)

        return s_l


In [9]:
import sys
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
from torch_geometric.typing import Adj, OptPairTensor, Size, SparseTensor
import torch
import torch.nn as nn
import matrix_util as Mat_Util
import Batched_GraphSage_Layer as batched_graphsage_layer
class Batched_DiffPool_Assignment_Layer(nn.Module):
    '''
    #    S, new clusters, new number of nodes
    '''
    def __init__(self, input_dim, assignment_num_block_layers, hid_dim, assigned_dim, Bias, normalize_graphsage, dropout,
                 aggregation, concat):
        super(Batched_DiffPool_Assignment_Layer, self).__init__()
        self.input_dim = input_dim
        self.assignment_num_block_layers = assignment_num_block_layers
        self.hid_dim = hid_dim
        self.assigned_dim = assigned_dim
        self.assignment_prediction_layer_input_dim = assigned_dim
        #########################################################.  General Parameters
        self.Bias = Bias
        self.normalize_graphsage = normalize_graphsage
        self.dropout = dropout
        self.aggregation = aggregation
        self.act_fun = F.relu
        self.concat=concat

        self.diffPool_assignment = nn.ModuleList()

        self.diffPool_assignment.append(GNN_Batched_GraphSage_Layer(input_dim=self.input_dim, output_dim=self.hid_dim,
                                                                                            Bias=self.Bias, normalize_graphsage=self.normalize_graphsage,
                                                                                            dropout=self.dropout, aggregation=self.aggregation, concat=self.concat))

        for i in range(assignment_num_block_layers):
            self.diffPool_assignment.append(GNN_Batched_GraphSage_Layer(input_dim=self.hid_dim, output_dim=self.hid_dim,
                                                                                                Bias=self.Bias, normalize_graphsage=self.normalize_graphsage,
                                                                                                dropout=self.dropout, aggregation=self.aggregation, concat=self.concat))

        self.diffPool_assignment.append(GNN_Batched_GraphSage_Layer(input_dim=self.hid_dim, output_dim=self.assigned_dim,
                                                                                            Bias=self.Bias, normalize_graphsage=self.normalize_graphsage,
                                                                                            dropout=self.dropout, aggregation=self.aggregation, concat=self.concat))


    def forward(self, input_tensor, tilda_adjacency_matrix):

        for layer in self.diffPool_assignment:
            input_tensor = self.act_fun(layer(input_tensor, tilda_adjacency_matrix))

        dense_prediction = F.softmax(input_tensor, dim=-1)

        return dense_prediction


In [ ]:

class DiffPool_Layer(nn.Module):
    def __init__(self, input_dim_size, new_feat_dim_size, new_num_nodes, Bias, normalize_graphsage, dropout, aggregation):
        super(DiffPool_Layer, self).__init__()
        self.input_dim_size = input_dim_size
        self.new_feat_dim_size = new_feat_dim_size
        self.new_num_nodes = new_num_nodes
        self.Bias = Bias
        self.normalize_graphsage = normalize_graphsage
        self.dropout = dropout
        self.aggregation = aggregation

        self.new_embed = DiffPool_Embedding_Layer(input_dim_size=input_dim_size, new_feat_dim_size=new_feat_dim_size, Bias=self.Bias, normalize_graphsage=self.normalize_graphsage, dropout=self.dropout, aggregation=self.aggregation)
        self.new_assign = DiffPool_Assignment_Layer(input_dim_size=self.input_dim_size, new_num_nodes=self.new_num_nodes, Bias=self.Bias , normalize_graphsage=self.normalize_graphsage , dropout=self.dropout, aggregation=self.aggregation)


    def forward(self, input_tensor, tilda_adjacency_matrix):
        #x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y

        z_l = self.new_embed(input_tensor, tilda_adjacency_matrix)
        s_l = self.new_assign(input_tensor, tilda_adjacency_matrix)

        new_X = torch.mm(s_l.transpose(-1, -2), z_l)
        new_adjacency = (s_l.transpose(-1, -2)).mm(tilda_adjacency_matrix).mm(s_l)

        return new_X, new_adjacency


In [10]:
import sys
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
from torch_geometric.typing import Adj, OptPairTensor, Size, SparseTensor
import torch
import torch.nn as nn
import matrix_util as Mat_Util
import Batched_DIFFPOOL_Assignment as batched_diffpool_assignment
import Batched_DIFFPOOL_Embedding as batched_diffpool_embedding
class Batched_DiffPool_Layer(nn.Module):
    def __init__(self, embedding_input_dim, embedding_num_block_layers, embedding_hid_dim, embedding_output_dim, assignment_input_dim,
                 assignment_num_block_layers, assignment_hid_dim, assignment_output_dim, concat, Weight_Initializer, Bias,
                 dropout_rate, normalize_graphsage, aggregation, act_fun):
        super(Batched_DiffPool_Layer, self).__init__()
        ################################################.  General Parameters
        self.Bias = Bias
        self.dropout_rate = dropout_rate
        self.aggregation = aggregation
        self.Weight_Initializer = Weight_Initializer
        self.concat = concat
        self.normalize_graphsage = normalize_graphsage

        ################################################.  Embedding and Assignment

        self.embedding_input_dim = embedding_input_dim
        self.embedding_num_block_layers = embedding_num_block_layers
        self.embedding_hid_dim = embedding_hid_dim
        self.embedding_output_dim = embedding_output_dim

        self.assignment_input_dim = assignment_input_dim
        self.assignment_num_block_layers = assignment_num_block_layers
        self.assignment_hid_dim = assignment_hid_dim
        self.assignment_output_dim = assignment_output_dim      # new number of nodes

        ################################################.  Embedding
        self.diffpool_embedding = Batched_DiffPool_Embedding_Layer(input_dim=self.embedding_input_dim,
                                                                                              embedding_num_block_layers=self.embedding_num_block_layers,
                                                                                              hid_dim=self.embedding_hid_dim,
                                                                                              embedded_dim=self.embedding_output_dim,
                                                                                              concat=self.concat,Bias=self.Bias,
                                                                                              normalize_graphsage=self.normalize_graphsage,
                                                                                              dropout=self.dropout_rate,
                                                                                              aggregation=self.aggregation)

        ################################################.  Assignment
        self.diffpool_assignment = Batched_DiffPool_Assignment_Layer(input_dim=self.assignment_input_dim,
                                                                                                 assignment_num_block_layers=self.assignment_num_block_layers,
                                                                                                 hid_dim=self.assignment_hid_dim,
                                                                                                 assigned_dim=self.assignment_output_dim,
                                                                                                 concat=self.concat, Bias=self.Bias,
                                                                                                 normalize_graphsage=self.normalize_graphsage,
                                                                                                 dropout=self.dropout_rate,
                                                                                                 aggregation=self.aggregation)
    def forward(self, features, adjacecny):
        features = features.to(torch.float32)

        embedding_output = self.diffpool_embedding(features, adjacecny)

        assignment_output = self.diffpool_assignment(features, adjacecny)

        #features = torch.matmul(torch.transpose(assignment_output, 1, 2), embedding_output)
        #adjacecny = torch.transpose(assignment_output, 1, 2) @ adjacecny @ assignment_output



        return embedding_output, assignment_output


In [ ]:
batched_diffpool_layer_example = Batched_DiffPool_Layer(embedding_input_dim=7, embedding_num_block_layers=1, embedding_hid_dim=7, embedding_output_dim=7,
                                                        assignment_input_dim=7, assignment_num_block_layers=1, assignment_hid_dim=7,
                                                        assignment_output_dim=7, Weight_Initializer=1, Bias=True, dropout_rate=0,
                                                        normalize_graphsage=True, concat=True, aggregation='mean', act_fun='ReLu')

In [ ]:
from torch_geometric.loader import DataLoader
batched_dataset = DataLoader(dataset, batch_size=30, shuffle=False)
for batched_graph in batched_dataset:
    x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y

    new_X, new_adjacency = batched_diffpool_layer_example(batched_graph, None)
    print("softmaxed_output: ", new_X.size())
    print("softmaxed_output: ", new_adjacency.size())
    break

TypeError: ignored

In [ ]:
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
from termcolor import colored

In [23]:
from torch_geometric.utils import dropout
from torch_geometric.loader import DataLoader
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
from torch_geometric.utils.train_test_split_edges import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
import sys
from torch_geometric.datasets import TUDataset
from scipy.sparse import csr_matrix
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
import Batched_GraphSage_Layer as batched_graphsage_layer
import Batched_DIFFPOOL_Assignment as batched_diffpool_assignment
import Batched_DIFFPOOL_Embedding as batched_diffpool_embedding
import Batched_DIFFPOOL_Layer as batched_diffpool_layer



class GlobalMeanPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return gnn.global_mean_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class DIFFPOOL_Model(nn.Module):
    '''
        DIFFPOOL Mode
    '''
    def __init__(self, embedding_input_dim, embedding_num_block_layers, embedding_hid_dim, new_feature_size, assignment_input_dim,
                 assignment_num_block_layers, assignment_hid_dim, max_number_of_nodes, concat_neighborhood, prediction_hid_layers,
                 num_classes, Weight_Initializer, Bias, dropout_rate, normalize_graphsage, aggregation, act_fun,
                 concat_diffpools_outputs, num_pooling, pooling):

        super(DIFFPOOL_Model, self).__init__()
        self.Bias = Bias
        self.dropout_rate = dropout_rate
        self.aggregation = aggregation
        self.act_fun = act_fun
        self.Weight_Initializer = Weight_Initializer
        self.concat_diffpools_outputs = concat_diffpools_outputs
        self.num_pooling = num_pooling
        self.concat = concat_neighborhood
        self.pooling = pooling


        self.embedding_input_dim = embedding_input_dim
        self.embedding_num_block_layers = embedding_num_block_layers
        self.embedding_hid_dim = embedding_hid_dim
        self.embedding_output_dim = new_feature_size
        self.normalize_graphsage = normalize_graphsage

        self.assignment_input_dim = assignment_input_dim
        self.assignment_num_block_layers = assignment_num_block_layers
        self.assignment_hid_dim = assignment_hid_dim
        self.max_number_of_nodes = max_number_of_nodes
        self.assignment_output_dim = int(self.max_number_of_nodes * 0.25)


        prediction_input_dim_sum = 0

        ###################################################.    DiffPool Layers
        diffpool_layers = []
        for i in range(self.num_pooling):
            diffpool_layers.append(Batched_DiffPool_Layer(embedding_input_dim=self.embedding_input_dim,
                                                                                 embedding_num_block_layers=self.embedding_num_block_layers,
                                                                                 embedding_hid_dim=self.embedding_hid_dim,
                                                                                 embedding_output_dim=self.embedding_output_dim,
                                                                                 assignment_input_dim=self.assignment_input_dim,
                                                                                 assignment_num_block_layers=self.assignment_num_block_layers,
                                                                                 assignment_hid_dim=self.assignment_hid_dim,
                                                                                 assignment_output_dim=self.assignment_output_dim,
                                                                                 concat=self.concat, Weight_Initializer=self.Weight_Initializer,
                                                                                 Bias=self.Bias, dropout_rate=self.dropout_rate,
                                                                                 normalize_graphsage=self.normalize_graphsage,
                                                                                 aggregation=self.aggregation, act_fun=self.act_fun))

            self.assignment_output_dim = int(self.assignment_output_dim * .25)
            self.embedding_input_dim = self.embedding_output_dim
            self.assignment_input_dim = self.embedding_output_dim
            prediction_input_dim_sum = prediction_input_dim_sum + self.embedding_output_dim
        self.diffpool_layers = nn.Sequential(*diffpool_layers)

        ###################################################.    Last Extra Embedding

        self.last_extra_embedding = Batched_DiffPool_Embedding_Layer(input_dim=self.embedding_output_dim,
                                                                                                embedding_num_block_layers=self.embedding_num_block_layers,
                                                                                                hid_dim=self.embedding_hid_dim, concat=self.concat,
                                                                                                embedded_dim=self.embedding_output_dim, Bias=self.Bias,
                                                                                                normalize_graphsage=self.normalize_graphsage, dropout=self.dropout_rate,
                                                                                                aggregation=self.aggregation)
        prediction_input_dim_sum = prediction_input_dim_sum + self.embedding_output_dim

        ###################################################.    Predictions
        self.prediction_input_dim = self.embedding_output_dim
        self.prediction_hid_layers = prediction_hid_layers
        self.num_classes = num_classes

        prediction_layers = []
        if len(self.prediction_hid_layers) == 0:
            if self.concat_diffpools_outputs:
                prediction_layers.append(nn.Linear(prediction_input_dim_sum, self.num_classes))
                self.prediction_model = nn.Sequential(*prediction_layers)
            else:
                prediction_layers.append(nn.Linear(self.prediction_input_dim, self.num_classes))
                self.prediction_model = nn.Sequential(*prediction_layers)
        else:
            if self.concat_diffpools_outputs:
                predict_input_dim = prediction_input_dim_sum
                for i in range(len(self.prediction_hid_layers)):
                    prediction_layers.append(nn.Linear(predict_input_dim, prediction_hid_layers[i]))
                    predict_input_dim = prediction_hid_layers[i]
                prediction_layers.append(nn.Linear(predict_input_dim, self.num_classes))
                self.prediction_model = nn.Sequential(*prediction_layers)
            else:
                predict_input_dim = self.prediction_input_dim
                for i in range(len(self.prediction_hid_layers)):
                    prediction_layers.append(nn.Linear(predict_input_dim, prediction_hid_layers[i]))
                    predict_input_dim = prediction_hid_layers[i]
                prediction_layers.append(nn.Linear(predict_input_dim, self.num_classes))
                self.prediction_model = nn.Sequential(*prediction_layers)

        if act_fun == 'ReLu':
            self.act_fun = F.relu
            print('ReLu is Selected.')
        elif act_fun == 'eLu':
            self.act_fun = nn.functional.elu
            print('eLu is Selected.')
        elif act_fun == 'tanh':
            self.act_fun = torch.tanh
            print('tanh is Selected.')
        self.act_fun_softmax = F.softmax

        mean = 0
        std = 0.1
        self.initialize_weights(Weight_Initializer, Bias, mean, std)

    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)

        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i, module in enumerate(model.children()):
                #print(i, module)
                if isinstance(module, torch.nn.Sequential):
                    for j, module_sub in enumerate(module):
                        #print("j: ",j,module_sub)
                        if isinstance(module_sub, batched_diffpool_layer.Batched_DiffPool_Layer):
                            #print(module_sub)
                            for party in module_sub.children():
                                if isinstance(party, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                                    for diff_embd_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_embd_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_embed_party_in_modulelist in diff_embd_party.children():
                                                if isinstance(diff_embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.xavier_normal_(diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    #torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_embed_party_in_modulelist.learnable_weights.weight: ",diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_embed_party_in_modulelist.learnable_weights.bias)
                                elif isinstance(party, batched_diffpool_assignment.Batched_DiffPool_Assignment_Layer):
                                    for diff_assign_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_assign_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_assign_party_in_modulelist in diff_assign_party.children():
                                                if isinstance(diff_assign_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.xavier_normal_(diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_assign_party_in_modulelist.learnable_weights.weight: ",diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_assign_party_in_modulelist.learnable_weights.bias)
                                        elif isinstance(diff_assign_party, torch.nn.Sequential):
                                            for diff_assign_party_in_sequential in diff_assign_party:
                                                if isinstance(diff_assign_party_in_sequential, torch.nn.Linear):
                                                    torch.nn.init.xavier_normal_(diff_assign_party_in_sequential.weight)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_sequential.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_sequential.bias)
                                                        #print(diff_assign_party_in_sequential.bias)
                        elif isinstance(module_sub, torch.nn.Linear):
                            #print("linear final prediction layers")
                            torch.nn.init.xavier_normal_(module_sub.weight)
                            if Bias:
                                torch.nn.init.zeros_(module_sub.bias)
                                #print(module_sub.bias)
                elif isinstance(module, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                    for embd_party in module.children():
                        #print("embd_party: ")#, embd_party)
                        if isinstance(embd_party, torch.nn.ModuleList):
                            for embed_party_in_modulelist in embd_party.children():
                                if isinstance(embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                    torch.nn.init.xavier_normal_(embed_party_in_modulelist.learnable_weights.weight)
                                    #torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.weight)
                                    #print("embed_party_in_modulelist.learnable_weights.weight: ",embed_party_in_modulelist.learnable_weights.weight)
                                    if Bias:
                                        torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.bias)
                                        #print(embed_party_in_modulelist.learnable_weights.bias)

        if Weight_Initializer == 2:                                             #.      1. Kaiming Normal_.
            for i, module in enumerate(model.children()):
                #print(i, module)
                if isinstance(module, torch.nn.Sequential):
                    for j, module_sub in enumerate(module):
                        #print("j: ",j,module_sub)
                        if isinstance(module_sub, batched_diffpool_layer.Batched_DiffPool_Layer):
                            #print(module_sub)
                            for party in module_sub.children():
                                if isinstance(party, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                                    for diff_embd_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_embd_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_embed_party_in_modulelist in diff_embd_party.children():
                                                if isinstance(diff_embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.kaiming_normal_(diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    #torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_embed_party_in_modulelist.learnable_weights.weight: ",diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_embed_party_in_modulelist.learnable_weights.bias)
                                elif isinstance(party, batched_diffpool_assignment.Batched_DiffPool_Assignment_Layer):
                                    for diff_assign_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_assign_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_assign_party_in_modulelist in diff_assign_party.children():
                                                if isinstance(diff_assign_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.kaiming_normal_(diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_assign_party_in_modulelist.learnable_weights.weight: ",diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_assign_party_in_modulelist.learnable_weights.bias)
                                        elif isinstance(diff_assign_party, torch.nn.Sequential):
                                            for diff_assign_party_in_sequential in diff_assign_party:
                                                if isinstance(diff_assign_party_in_sequential, torch.nn.Linear):
                                                    torch.nn.init.kaiming_normal_(diff_assign_party_in_sequential.weight)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_sequential.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_sequential.bias)
                                                        #print(diff_assign_party_in_sequential.bias)
                        elif isinstance(module_sub, torch.nn.Linear):
                            #print("linear final prediction layers")
                            torch.nn.init.kaiming_normal_(module_sub.weight)
                            if Bias:
                                torch.nn.init.zeros_(module_sub.bias)
                                #print(module_sub.bias)
                elif isinstance(module, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                    for embd_party in module.children():
                        #print("embd_party: ")#, embd_party)
                        if isinstance(embd_party, torch.nn.ModuleList):
                            for embed_party_in_modulelist in embd_party.children():
                                if isinstance(embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                    torch.nn.init.kaiming_normal_(embed_party_in_modulelist.learnable_weights.weight)
                                    #torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.weight)
                                    #print("embed_party_in_modulelist.learnable_weights.weight: ",embed_party_in_modulelist.learnable_weights.weight)
                                    if Bias:
                                        torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.bias)
                                        #print(embed_party_in_modulelist.learnable_weights.bias)

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i, module in enumerate(model.children()):
                #print(i, module)
                if isinstance(module, torch.nn.Sequential):
                    for j, module_sub in enumerate(module):
                        #print("j: ",j,module_sub)
                        if isinstance(module_sub, batched_diffpool_layer.Batched_DiffPool_Layer):
                            #print(module_sub)
                            for party in module_sub.children():
                                if isinstance(party, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                                    for diff_embd_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_embd_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_embed_party_in_modulelist in diff_embd_party.children():
                                                if isinstance(diff_embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.normal_(diff_embed_party_in_modulelist.learnable_weights.weight, mean=mean, std=std)
                                                    #torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_embed_party_in_modulelist.learnable_weights.weight: ",diff_embed_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_embed_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_embed_party_in_modulelist.learnable_weights.bias)
                                elif isinstance(party, batched_diffpool_assignment.Batched_DiffPool_Assignment_Layer):
                                    for diff_assign_party in party.children():
                                        #print("diff_embd_party: ")#, diff_embd_party)
                                        if isinstance(diff_assign_party, torch.nn.ModuleList):
                                            #print("moduel list for diffpool embed")
                                            for diff_assign_party_in_modulelist in diff_assign_party.children():
                                                if isinstance(diff_assign_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                                    torch.nn.init.normal_(diff_assign_party_in_modulelist.learnable_weights.weight, mean=mean, std=std)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    #print("diff_assign_party_in_modulelist.learnable_weights.weight: ",diff_assign_party_in_modulelist.learnable_weights.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_modulelist.learnable_weights.bias)
                                                        #print(diff_assign_party_in_modulelist.learnable_weights.bias)
                                        elif isinstance(diff_assign_party, torch.nn.Sequential):
                                            for diff_assign_party_in_sequential in diff_assign_party:
                                                if isinstance(diff_assign_party_in_sequential, torch.nn.Linear):
                                                    torch.nn.init.normal_(diff_assign_party_in_sequential.weight, mean=mean, std=std)
                                                    #torch.nn.init.zeros_(diff_assign_party_in_sequential.weight)
                                                    if Bias:
                                                        torch.nn.init.zeros_(diff_assign_party_in_sequential.bias)
                                                        #print(diff_assign_party_in_sequential.bias)
                        elif isinstance(module_sub, torch.nn.Linear):
                            #print("linear final prediction layers")
                            torch.nn.init.normal_(module_sub.weight, mean=mean, std=std)
                            if Bias:
                                torch.nn.init.zeros_(module_sub.bias)
                                #print(module_sub.bias)
                elif isinstance(module, batched_diffpool_embedding.Batched_DiffPool_Embedding_Layer):
                    for embd_party in module.children():
                        #print("embd_party: ")#, embd_party)
                        if isinstance(embd_party, torch.nn.ModuleList):
                            for embed_party_in_modulelist in embd_party.children():
                                if isinstance(embed_party_in_modulelist, batched_graphsage_layer.GNN_Batched_GraphSage_Layer):
                                    torch.nn.init.normal_(embed_party_in_modulelist.learnable_weights.weight, mean=mean, std=std)
                                    #torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.weight)
                                    #print("embed_party_in_modulelist.learnable_weights.weight: ",embed_party_in_modulelist.learnable_weights.weight)
                                    if Bias:
                                        torch.nn.init.zeros_(embed_party_in_modulelist.learnable_weights.bias)
                                        #print(embed_party_in_modulelist.learnable_weights.bias)







    def computational_matricess(self, batched_graphs, edge_mask):
        if edge_mask == None:
            joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
        else:
            joint_tilda_adjacency_matrix = torch.tensor(csr_matrix((np.array(edge_mask), (np.array(batched_graphs.edge_index[0]), np.array(batched_graphs.edge_index[1])))).todense())

        joint_tilda_adjacency_matrix = joint_tilda_adjacency_matrix.type(torch.float32)
        #batch_size = batched_graphs.num_graphs


        if batched_graphs.batch is not None:
            graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
            batch_size = batched_graphs.num_graphs
        else:
            graph_sizes = [len(batched_graphs.x)]
            batch_size = 1
        max_number_of_nodes_in_batch_of_graphs = max(graph_sizes)
        #print("max_number_of_nodes_in_batch_of_graphs: ", max_number_of_nodes_in_batch_of_graphs)


        new_number_of_nodes = int(joint_tilda_adjacency_matrix.size()[0] / batch_size)

        adjacency_list = []
        feature_list = []
        for i in range(batch_size):
            start = i * graph_sizes[i]
            end = (i + 1) * graph_sizes[i]
            un_padded_adj = joint_tilda_adjacency_matrix[start:end, start:end]
            off_set = max_number_of_nodes_in_batch_of_graphs - un_padded_adj.size()[0]
            if un_padded_adj.size()[0] < max_number_of_nodes_in_batch_of_graphs:
                un_padded_adj = un_padded_adj.numpy()
                un_padded_adj = np.pad(un_padded_adj, [(0, off_set), (0, off_set)], mode='constant', constant_values=np.zeros(1,dtype=np.float32))
                un_padded_adj = torch.from_numpy(un_padded_adj)
            un_padded_adj = un_padded_adj.type(torch.float32)
            adjacency_list.append(un_padded_adj)

            un_padded_feat = batched_graphs.x[start:end, :].numpy()
            un_padded_feat = torch.from_numpy(np.pad(un_padded_feat, [(0, off_set), (0, 0)], mode='constant', constant_values=np.zeros(1,dtype=np.float32)))
            feature_list.append(un_padded_feat)


        adjacency_list = list(map(lambda x: torch.unsqueeze(x, 0), adjacency_list))
        feature_list = list(map(lambda x: torch.unsqueeze(x, 0), feature_list))

        new_adjacecny = torch.cat(adjacency_list, dim=0)
        new_features = torch.cat(feature_list, dim=0)



        return new_adjacecny, new_features

    def forward(self, batched_graphs, edge_mask):
        adjacecny, features = self.computational_matricess(batched_graphs, edge_mask)
        #print(adjacecny.size())
        concatination_list_of_poolings = []

        for i in range(self.num_pooling):
            embedding_output, assignment_output = self.diffpool_layers[i](features, adjacecny)
            #features = torch.matmul(torch.transpose(assignment_output, 1, 2), embedding_output)
            features = torch.bmm(torch.transpose(assignment_output, 1, 2), embedding_output)
            adjacecny = torch.transpose(assignment_output, 1, 2) @ adjacecny @ assignment_output


            if self.pooling == "max":
                embedding_output_pooled, _ = torch.max(embedding_output, dim=1)
            elif self.pooling == "mean":
                embedding_output_pooled = torch.mean(embedding_output, dim=1)
            elif self.pooling == "sum":
                embedding_output_pooled = torch.sum(embedding_output, dim=1)
            concatination_list_of_poolings.append(embedding_output_pooled)


        extra_embed = self.last_extra_embedding(features, adjacecny)
        if self.pooling == "max":
            extra_embed_pooled, _ = torch.max(extra_embed, dim=1)
        elif self.pooling == "mean":
            extra_embed_pooled = torch.mean(extra_embed, dim=1)
        elif self.pooling == "sum":
            extra_embed_pooled = torch.sum(extra_embed, dim=1)
        concatination_list_of_poolings.append(extra_embed_pooled)

        if self.concat_diffpools_outputs:
            output = torch.cat(concatination_list_of_poolings, dim=1)
        else:
            output = extra_embed_pooled

        prediction_output = output
        for i in range(len(self.prediction_hid_layers)):

            prediction_output = self.act_fun(self.prediction_model[i](prediction_output))
        prediction_output = F.softmax(self.prediction_model[-1](prediction_output), dim=-1)



        return concatination_list_of_poolings, prediction_output




dataset = TUDataset(root='data/TUDataset', name='MUTAG')
sum=0
length_list = []
for i in range(len(dataset)):
    sum = sum + len(dataset[i].x)
    length_list.append(len(dataset[i].x))

#print("max number of nodes in graphs: ", max(length_list))
batch_size = 10


node_feat_size = len(dataset[0].x[0])

hid_dim = 7

diffpool_model_example = DIFFPOOL_Model(embedding_input_dim=7, embedding_num_block_layers=1, embedding_hid_dim=64, new_feature_size=64,
                                    assignment_input_dim=7, assignment_num_block_layers=1, assignment_hid_dim=64, max_number_of_nodes=256,
                                    prediction_hid_layers=[50], concat_neighborhood=False, num_classes=2, Weight_Initializer=3,
                                    Bias=True, dropout_rate=0, normalize_graphsage=False, aggregation="mean",
                                    act_fun="ReLu", concat_diffpools_outputs=True, num_pooling=1, pooling="max")
#print(diffpool_model_example)
batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=False)

for batched_graph in batched_dataset:
    x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y
    concatination_list_of_poolings, prediction_output = diffpool_model_example(batched_graph, None)
    print("Final Output: ", prediction_output)
    #print("concatination_list_of_poolings: ", concatination_list_of_poolings)
    break




ReLu is Selected.
Final Output:  tensor([[0.5171, 0.4829],
        [0.5177, 0.4823],
        [0.5178, 0.4822],
        [0.5187, 0.4813],
        [0.5155, 0.4845],
        [0.5178, 0.4822],
        [0.5188, 0.4812],
        [0.5180, 0.4820],
        [0.5190, 0.4810],
        [0.5177, 0.4823]], grad_fn=<SoftmaxBackward0>)


In [24]:
for batched_graph in batched_dataset:
    x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y
    tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense())))
    #print("tilda_adjacency_matrix: ", tilda_adjacency_matrix.size())
    gnn_sage1_output = diffpool_model_example(batched_graph, None)
    #print("gnn_sage1_output: ", gnn_sage1_output.size())
    print(gnn_sage1_output)
    #break

([tensor([[0.0816, 0.0000, 0.0591, 0.2343, 0.0000, 0.0312, 0.0000, 0.1246, 0.0860,
         0.0000, 0.0000, 0.0518, 0.0464, 0.0541, 0.0996, 0.0000, 0.0000, 0.0665,
         0.0000, 0.0000, 0.1304, 0.0000, 0.0636, 0.0000, 0.0198, 0.0842, 0.0000,
         0.0000, 0.0714, 0.0149, 0.0250, 0.1109, 0.0662, 0.2061, 0.0000, 0.0399,
         0.1673, 0.0000, 0.1367, 0.0000, 0.1121, 0.0000, 0.0864, 0.0000, 0.1172,
         0.1335, 0.0366, 0.0000, 0.1946, 0.0000, 0.0113, 0.0000, 0.0399, 0.0000,
         0.0000, 0.0000, 0.2024, 0.0000, 0.0000, 0.0852, 0.0000, 0.1726, 0.1025,
         0.1216],
        [0.0845, 0.0000, 0.0559, 0.2329, 0.0000, 0.0422, 0.0000, 0.1224, 0.0921,
         0.0000, 0.0000, 0.0510, 0.0709, 0.0541, 0.0996, 0.0000, 0.0000, 0.0665,
         0.0000, 0.0000, 0.1260, 0.0000, 0.0636, 0.0000, 0.0198, 0.1124, 0.0000,
         0.0000, 0.0872, 0.0170, 0.0224, 0.1109, 0.0635, 0.1988, 0.0000, 0.0275,
         0.1677, 0.0000, 0.1555, 0.0000, 0.1194, 0.0000, 0.0864, 0.0000, 0.1267,
        

In [25]:
output_diffpool = diffpool_model_example(dataset[2], None)
print(output_diffpool)

([tensor([[0.0768, 0.0000, 0.0510, 0.2335, 0.0000, 0.0314, 0.0000, 0.1239, 0.0850,
         0.0000, 0.0000, 0.0278, 0.0465, 0.0043, 0.0891, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1145, 0.0000, 0.0460, 0.0000, 0.0000, 0.0846, 0.0000,
         0.0000, 0.0712, 0.0026, 0.0225, 0.0947, 0.0606, 0.2051, 0.0000, 0.0403,
         0.1530, 0.0000, 0.1372, 0.0000, 0.1122, 0.0000, 0.0773, 0.0000, 0.1174,
         0.1331, 0.0192, 0.0000, 0.1844, 0.0000, 0.0000, 0.0000, 0.0281, 0.0000,
         0.0000, 0.0000, 0.2012, 0.0000, 0.0000, 0.0737, 0.0000, 0.1710, 0.0610,
         0.1221]], grad_fn=<MaxBackward0>), tensor([[0.0384, 0.0000, 0.1117, 0.0000, 0.0582, 0.0877, 0.0000, 0.0000, 0.0597,
         0.0000, 0.0000, 0.1274, 0.0350, 0.0947, 0.0383, 0.1314, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0826, 0.1143, 0.0000, 0.0607, 0.0000, 0.0000, 0.1090,
         0.0000, 0.0570, 0.0993, 0.0000, 0.0061, 0.0887, 0.0000, 0.0290, 0.0000,
         0.0000, 0.0883, 0.0291, 0.0269, 0.0387, 0.0836, 0.0000

In [28]:
GNN_Model_optimizer = torch.optim.Adam(diffpool_model_example.parameters(), lr=0.01, weight_decay=1e-14)

In [29]:
criterion = torch.nn.CrossEntropyLoss()
def loss_calculations(preds, gtruth):
    loss_per_epoch = criterion(preds, gtruth)
    return loss_per_epoch

In [30]:
def loading_model(load_index):
    GNN_Model = diffpool_model.DIFFPOOL_Model(embedding_input_dim=7, embedding_num_block_layers=1, embedding_hid_dim=64,
                                              new_feature_size=64, assignment_input_dim=7, assignment_num_block_layers=1,
                                              assignment_hid_dim=64, max_number_of_nodes=256, prediction_hid_layers=[50],
                                              concat_neighborhood=False, num_classes=2, Weight_Initializer=1, Bias=classifier_bias,
                                              dropout_rate=0, normalize_graphsage=False, aggregation="mean", act_fun="ReLu",
                                              concat_diffpools_outputs=True, num_pooling=1, pooling="mean")
    GNN_Model_optimizer = torch.optim.Adam(GNN_Model.parameters(), lr=classifier_lr, weight_decay=classifier_weight_decay)
    checkpoint = torch.load("/content/drive/My Drive/Explainability Methods/"+str(Explainability_name) + " on " + str(Task_name) + "/Model/" + File_Name + str(epoch + load_index + 1) + ".pt")
    GNN_Model.load_state_dict(checkpoint['model_state_dict'])
    GNN_Model_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    return GNN_Model, GNN_Model_optimizer, epoch

In [33]:
def visualize_losses(g_losses, epoch_history):
    g_losses_list = torch.stack(g_losses).cpu().detach().numpy()

    fig = plt.figure(figsize=(27,20))

    ax = plt.subplot2grid((3, 1), (0, 0), colspan=1)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title("SA Model Loss in Epoch: " + str(epoch_history))

    ax.plot(g_losses_list, color='r')

    #plt.savefig('/content/drive/My Drive/Explainability Methods/'+str(Explainability_name)+' on ' + str(Task_name) + '/Experimental Results/' + File_Name + 'Loss_til_epoch_{:04d}.png'.format(epoch_history))
    plt.show()




In [34]:
def train_step():
    GNN_Model_loss_batch = []
    Pred_Labels = []
    Real_Labels = []

    GNN_Model.train()
    GNN_Model.zero_grad()
    #torch.autograd.set_detect_anomaly(True)
    for batch_of_graphs in train_loader:

        concatination_list_of_poolings, softmaxed_h2 = GNN_Model(batch_of_graphs, None)
        batch_loss = loss_calculations(softmaxed_h2, batch_of_graphs.y)


        Pred_Labels.extend(softmaxed_h2.argmax(dim=1).detach().tolist())
        Real_Labels.extend(batch_of_graphs.y.detach().tolist())
        GNN_Model_loss_batch.append(batch_loss)

        batch_loss.backward()

        GNN_Model_optimizer.step()

    return torch.mean(torch.tensor(GNN_Model_loss_batch)), metrics.accuracy_score(Real_Labels, Pred_Labels)


In [ ]:
def batching_adjacency(whole_graphs_adjacency, batch_feat, batch_size):
    """
        transform a batched graph to batched adjacency tensor and node feature tensor
    """
    #print("whole_graphs_adjacency.size()[0]: ", whole_graphs_adjacency.size()[0])
    new_number_of_nodes = int(whole_graphs_adjacency.size()[0] / batch_size)
    #print(batch_size)
    adjacency_list = []
    feature_list = []
    for i in range(batch_size):
        start = i * new_number_of_nodes
        end = (i + 1) * new_number_of_nodes
        adjacency_list.append(whole_graphs_adjacency[start:end, start:end])
        feature_list.append(batch_feat[start:end, :])
    adjacency_list = list(map(lambda x: torch.unsqueeze(x, 0), adjacency_list))
    feature_list = list(map(lambda x: torch.unsqueeze(x, 0), feature_list))
    new_adjacecny = torch.cat(adjacency_list, dim=0)
    new_features = torch.cat(feature_list, dim=0)

    return new_features, new_adjacecny

In [ ]:
batch_size = 32
batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=False)

for batched_graph in batched_dataset:
    x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y
    tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graph.edge_index).todense())))
    feat,adj = batching_adjacency(tilda_adjacency_matrix, x, batch_size)
    break
print(adj.size())
print(feat.size())

whole_graphs_adjacency.size()[0]:  585
32
torch.Size([32, 18, 18])
torch.Size([32, 18, 7])


In [ ]:
        #tilda_degree_vector = torch.sum(tilda_adjacency_matrix, dim=1)

        #k = tilda_degree_vector.size(0)
        #tilda_degree_matrix = torch.zeros(k, k)
        #tilda_degree_matrix.as_strided([k], [k + 1]).copy_(tilda_degree_vector)
        #tilda_degree_matrix = tilda_degree_matrix.to(torch.float32)

        #ones_matrix = torch.ones_like(tilda_degree_matrix)

        #reciprocal_tilda_degree_matrix = ones_matrix.div(tilda_degree_matrix)
        #reciprocal_tilda_degree_matrix = torch.nan_to_num(reciprocal_tilda_degree_matrix, nan=0, posinf=0, neginf=0)

        #return tilda_adjacency_matrix, reciprocal_tilda_degree_matrix
        #tilda_degree_matrix = torch.nan_to_num(tilda_degree_matrix, nan=0, posinf=0, neginf=0)

In [ ]:
def diffpool_new_number_of_nodes(your_dataset):
    size_list = []
    for graph in your_dataset:
        size_list.append(len(graph.x))
    return max(size_list)
print(diffpool_new_number_of_nodes(dataset))

28


In [ ]:
def computational_matricess(batched_graphs, edge_mask):
    if edge_mask == None:
        joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
    else:
        joint_tilda_adjacency_matrix = torch.tensor(csr_matrix((np.array(edge_mask), (np.array(batched_graphs.edge_index[0]), np.array(batched_graphs.edge_index[1])))).todense())
    #joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense()) + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
    joint_tilda_adjacency_matrix = joint_tilda_adjacency_matrix.type(torch.float32)
    batch_size = batched_graphs.num_graphs


    if batched_graphs.batch is not None:
        graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
    else:
        graph_sizes = [len(batched_graphs.x)]
    max_number_of_nodes_in_batch_of_graphs = max(graph_sizes)
    print("max_number_of_nodes_in_batch_of_graphs: ", max_number_of_nodes_in_batch_of_graphs)


    #print("whole_graphs_adjacency.size()[0]: ", whole_graphs_adjacency.size()[0])
    new_number_of_nodes = int(joint_tilda_adjacency_matrix.size()[0] / batch_size)

    adjacency_list = []
    feature_list = []
    for i in range(batch_size):
        start = i * graph_sizes[i]
        end = (i + 1) * graph_sizes[i]
        un_padded_adj = joint_tilda_adjacency_matrix[start:end, start:end]
        off_set = max_number_of_nodes_in_batch_of_graphs - un_padded_adj.size()[0]
        if un_padded_adj.size()[0] < max_number_of_nodes_in_batch_of_graphs:
            un_padded_adj = un_padded_adj.numpy()
            un_padded_adj = np.pad(un_padded_adj, [(0, off_set), (0, off_set)], mode='constant')
            un_padded_adj = torch.from_numpy(un_padded_adj)
        adjacency_list.append(un_padded_adj)

        un_padded_feat = batched_graphs.x[start:end, :].numpy()
        un_padded_feat = torch.from_numpy(np.pad(un_padded_feat, [(0, off_set), (0, 0)], mode='constant'))
        feature_list.append(un_padded_feat)


    adjacency_list = list(map(lambda x: torch.unsqueeze(x, 0), adjacency_list))
    feature_list = list(map(lambda x: torch.unsqueeze(x, 0), feature_list))
    new_adjacecny = torch.cat(adjacency_list, dim=0)
    print("new_adjacecny: ", new_adjacecny.size())
    new_features = torch.cat(feature_list, dim=0)
    print("new_features: ", new_features.size())
    #new_adjacecny = new_adjacecny.view(batch_size, new_number_of_nodes, new_number_of_nodes)

    return new_adjacecny, new_features



batch_size = 10
batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=False)


for batched_graph in batched_dataset:
    x, edge_index, batch, y = batched_graph.x, batched_graph.edge_index, batched_graph.batch, batched_graph.y
    concatination_list_of_poolings, prediction_output = computational_matricess(batched_graph, None)
    print("Final Output: ", prediction_output)
    break

max_number_of_nodes_in_batch_of_graphs:  28
new_adjacecny:  torch.Size([10, 28, 28])
new_features:  torch.Size([10, 28, 7])
Final Output:  tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.

In [ ]:
import numpy as np
a = np.array([[ 1.,  1.,  1.,  1.,  1.],
               [ 1.,  1.,  1.,  1.,  1.],
               [ 1.,  1.,  1.,  1.,  1.]])
a = np.pad(a, [(0, 1), (0, 2)], mode='constant')
print(a)

[[1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
